In [1]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Latex

In [30]:
k1, k2, k3 = 1, 4, 1
p1, p2, p3 = 2, 1.2, 1
q1, q2, q3 = 1, 2, 1.5
m1, m2, m3 = 2, 3, 3

alpha1, alpha2 = 1.5, 1.1
a, b = 0, 4
n = 41 # 81, 201

In [21]:
x = sp.symbols('x')
f = sp.symbols('f', cls=sp.Function)(x)
u = m1 * sp.cos(m2 * x) + m3
k = k1 * sp.cos(k2 * x) + k3
p = p1 * sp.sin(p2 * x) + p3
q = q1 * sp.sin(q2 * x) + q3

In [22]:
eq0 = -sp.diff(k*sp.diff(f, x), x) + p * sp.diff(f, x) + q * f
eq1 = (-k * sp.diff(u, x) + alpha1 * u)
eq2 = (k * sp.diff(u, x) + alpha2 * u)

In [26]:
f1 = eq0.subs({f:u}).simplify()
f1

-6*(2*sin(1.2*x) + 1)*sin(3*x) + (sin(2*x) + 1.5)*(2*cos(3*x) + 3) + 18*(cos(4*x) + 1)*cos(3*x) - 24*sin(3*x)*sin(4*x)

In [28]:
C1 = eq1.subs({f:u}).simplify()
C1

6*(cos(4*x) + 1)*sin(3*x) + 3.0*cos(3*x) + 4.5

In [29]:
C2 = eq2.subs({f:u}).simplify()
C2

-6*(cos(4*x) + 1)*sin(3*x) + 2.2*cos(3*x) + 3.3

In [35]:
mu1 = sp.simplify(eq1.subs({u:u}).subs({x:a}))
mu1

7.50000000000000

In [34]:
mu2 = sp.simplify(eq2.subs({u:u}).subs({x:b})).evalf()
mu2

5.29279136636670

In [36]:
X = np.linspace(start=a, stop=b, num=n + 1)
h = (b - a) * 1.0 / n

In [40]:
sp.Integral(f1, (x, a, b)).doit().evalf()

13.2842999085157